Librerías

In [1]:
import pandas as pd
import numpy as np

Carga de datos

In [2]:
clientes = pd.read_csv('cliente_tabla.csv')
productos = pd.read_csv('producto_tabla.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test  = pd.read_csv('test.csv')
town_state  = pd.read_csv('town_state.csv')

In [3]:
train = pd.read_csv('train.csv')

In [10]:
productos.head()

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [9]:
train

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.00,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.00,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.00,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.00,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.00,3
...,...,...,...,...,...,...,...,...,...,...,...
74180459,9,25759,1,5517,4528866,32873,4,27.76,0,0.00,4
74180460,9,25759,1,5517,4528866,34226,4,37.04,0,0.00,4
74180461,9,25759,1,5517,4528866,45112,4,51.00,0,0.00,4
74180462,9,25759,1,5517,4547943,40217,3,26.94,3,26.94,0
